In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler,MaxAbsScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import *

In [2]:
numeric_cols=columns_to_use=['id', 'Rating', 'maincateg', 'platform', 'price1', 'actprice1', 'norating1', 'noreviews1', 'star_5f', 'star_4f', 'star_3f',
       'star_2f', 'star_1f', 'fulfilled1']

# Train Data

In [3]:
train=pd.read_csv('trainn.csv', usecols=numeric_cols)

In [4]:
train.head()

,id,Rating,maincateg,platform,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,3.9,Women,Flipkart,698,999,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,3.8,Men,Flipkart,999,1999,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,4.4,Women,Flipkart,2749,4999,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,4.2,Men,Flipkart,518,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,3.9,Men,Flipkart,1379,2299,77.0,3.0,35.0,21.0,7.0,7,7,1


In [5]:
train.isnull().sum()

id              0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
norating1     678
noreviews1    578
star_5f       588
star_4f       539
star_3f       231
star_2f         0
star_1f         0
fulfilled1      0
dtype: int64

In [6]:
dummy_columns = ['maincateg', 'platform']
for column in dummy_columns:
    temptrain = pd.get_dummies(train[column], prefix=column)
    train = pd.merge(
        left=train,
        right=temptrain,
        left_index=True,
        right_index=True,
    )
    train = train.drop(columns=column)
train.shape
train.head()

,id,Rating,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,maincateg_Men,maincateg_Women,platform_Amazon,platform_Flipkart
0,16695,3.9,698,999,38.0,7.0,17.0,9.0,6.0,3,3,0,0,1,0,1
1,5120,3.8,999,1999,531.0,69.0,264.0,92.0,73.0,29,73,1,1,0,0,1
2,18391,4.4,2749,4999,17.0,4.0,11.0,3.0,2.0,1,0,1,0,1,0,1
3,495,4.2,518,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1,1,0,0,1
4,16408,3.9,1379,2299,77.0,3.0,35.0,21.0,7.0,7,7,1,1,0,0,1


In [7]:
train.columns
main_cols=['id', 'Rating', 'price1', 'actprice1', 'norating1', 'noreviews1',
       'star_5f', 'star_4f', 'star_3f', 'star_2f', 'star_1f',
       'maincateg_Men', 'maincateg_Women', 'platform_Amazon',
       'platform_Flipkart', 'fulfilled1']

In [8]:
imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(train)

SimpleImputer()

In [9]:
train.mean()

id                   10479.541577
Rating                   4.012873
price1                 688.070693
actprice1             1369.286777
norating1             3057.660776
noreviews1             423.976307
star_5f               1585.239466
star_4f                655.923310
star_3f                357.260662
star_2f                155.085188
star_1f                275.500572
fulfilled1               0.601526
maincateg_Men            0.408328
maincateg_Women          0.558233
platform_Amazon          0.021678
platform_Flipkart        0.978322
dtype: float64

In [10]:
train=imputer.transform(train)
train_new=pd.DataFrame(train, columns=main_cols)
train_new

,id,Rating,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,maincateg_Men,maincateg_Women,platform_Amazon,platform_Flipkart,fulfilled1
0,16695.0,3.9,698.0,999.0,38.000000,7.0,17.000000,9.00000,6.0,3.0,3.0,0.0,0.0,1.0,0.0,1.0
1,5120.0,3.8,999.0,1999.0,531.000000,69.0,264.000000,92.00000,73.0,29.0,73.0,1.0,1.0,0.0,0.0,1.0
2,18391.0,4.4,2749.0,4999.0,17.000000,4.0,11.000000,3.00000,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,495.0,4.2,518.0,724.0,46413.000000,6229.0,1045.000000,12416.00000,5352.0,701.0,4595.0,1.0,1.0,0.0,0.0,1.0
4,16408.0,3.9,1379.0,2299.0,77.000000,3.0,35.000000,21.00000,7.0,7.0,7.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,16554.0,4.3,567.0,1199.0,807.000000,114.0,485.000000,177.00000,61.0,41.0,43.0,0.0,0.0,1.0,0.0,1.0
15726,19600.0,3.9,499.0,998.0,246.000000,34.0,120.000000,45.00000,37.0,16.0,28.0,1.0,0.0,1.0,0.0,1.0
15727,6733.0,3.7,329.0,499.0,3057.660776,18.0,1585.239466,655.92331,20.0,10.0,15.0,1.0,0.0,1.0,0.0,1.0
15728,3889.0,3.9,2249.0,4499.0,750.000000,479.0,13.000000,6.00000,10.0,25.0,47.0,1.0,1.0,0.0,1.0,0.0


In [11]:
train_new.isnull().mean()

id                   0.0
Rating               0.0
price1               0.0
actprice1            0.0
norating1            0.0
noreviews1           0.0
star_5f              0.0
star_4f              0.0
star_3f              0.0
star_2f              0.0
star_1f              0.0
maincateg_Men        0.0
maincateg_Women      0.0
platform_Amazon      0.0
platform_Flipkart    0.0
fulfilled1           0.0
dtype: float64

In [12]:
train_new.shape
train_new1=train_new.astype(int)

In [13]:
X_train=train_new1.drop(['price1', 'id', 'norating1', 'Rating',
       'star_5f', 'star_4f', 'star_3f', 'star_2f', 'star_1f'], axis=1)
y_train=train_new1['price1']

# Test Data

In [14]:
numeric_cols1=columns_to_use=['id', 'Rating', 'maincateg', 'platform', 'actprice1', 'norating1', 'noreviews1', 'star_5f', 'star_4f', 'star_3f',
       'star_2f', 'star_1f', 'fulfilled1']

In [15]:
test=pd.read_csv('test.csv', usecols=numeric_cols1)
test.head()

,id,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,2242,3.8,Men,Flipkart,999,27928,3543,14238.0,4295,3457,1962,3976.0,1
1,20532,3.9,Women,Flipkart,499,3015,404,1458.0,657,397,182,321.0,1
2,10648,3.9,Women,Flipkart,999,449,52,229.0,70,71,33,46.0,1
3,20677,3.9,Men,Flipkart,2999,290,40,141.0,51,49,17,32.0,1
4,12593,3.9,Men,Flipkart,999,2423,326,1265.0,414,293,143,308.0,0


In [16]:
test.isnull().sum()

id              0
Rating        203
maincateg      67
platform        0
actprice1       0
norating1       0
noreviews1      0
star_5f        68
star_4f         0
star_3f         0
star_2f         0
star_1f       186
fulfilled1      0
dtype: int64

In [17]:
dummy_columns = ['maincateg', 'platform']
for column in dummy_columns:
    temptest = pd.get_dummies(test[column], prefix=column)
    test = pd.merge(
        left=test,
        right=temptest,
        left_index=True,
        right_index=True,
    )
    test = test.drop(columns=column)
test.shape
test.head()

,id,Rating,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,maincateg_Men,maincateg_Women,platform_Amazon,platform_Flipkart
0,2242,3.8,999,27928,3543,14238.0,4295,3457,1962,3976.0,1,1,0,0,1
1,20532,3.9,499,3015,404,1458.0,657,397,182,321.0,1,0,1,0,1
2,10648,3.9,999,449,52,229.0,70,71,33,46.0,1,0,1,0,1
3,20677,3.9,2999,290,40,141.0,51,49,17,32.0,1,1,0,0,1
4,12593,3.9,999,2423,326,1265.0,414,293,143,308.0,0,1,0,0,1


In [18]:
test.shape
test.dtypes

id                     int64
Rating               float64
actprice1              int64
norating1              int64
noreviews1             int64
star_5f              float64
star_4f                int64
star_3f                int64
star_2f                int64
star_1f              float64
fulfilled1             int64
maincateg_Men          uint8
maincateg_Women        uint8
platform_Amazon        uint8
platform_Flipkart      uint8
dtype: object

In [19]:
main_cols1=['id','Rating', 'actprice1', 'norating1', 'noreviews1',
       'star_5f', 'star_4f', 'star_3f', 'star_2f', 'star_1f',
       'maincateg_Men', 'maincateg_Women', 'platform_Amazon',
       'platform_Flipkart', 'fulfilled1']

In [20]:
imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(test)
test=imputer.transform(test)
test_new=pd.DataFrame(test, columns=main_cols1)
test_new1=test_new.astype(int)

In [21]:
test_new.isnull().mean()

id                   0.0
Rating               0.0
actprice1            0.0
norating1            0.0
noreviews1           0.0
star_5f              0.0
star_4f              0.0
star_3f              0.0
star_2f              0.0
star_1f              0.0
maincateg_Men        0.0
maincateg_Women      0.0
platform_Amazon      0.0
platform_Flipkart    0.0
fulfilled1           0.0
dtype: float64

In [22]:
test_new.shape
X_test1=test_new.astype(int)
test_new.dtypes

id                   float64
Rating               float64
actprice1            float64
norating1            float64
noreviews1           float64
star_5f              float64
star_4f              float64
star_3f              float64
star_2f              float64
star_1f              float64
maincateg_Men        float64
maincateg_Women      float64
platform_Amazon      float64
platform_Flipkart    float64
fulfilled1           float64
dtype: object

In [23]:
X_test=X_test1.drop(['id', 'norating1', 'Rating',
       'star_5f'], axis=1)

# Standardisation and modelling

In [24]:
scaler = StandardScaler()

 #fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

 #transform train and test sets
#X_train_scale = scaler.transform(X_train)
#X_test_scale = scaler.transform(X_test)

StandardScaler()

In [25]:
from sklearn.metrics import mean_squared_error
rfr=RandomForestRegressor()
rfr.get_params()
#ss = {'n_estimators' : [100,200,500, 800, 1000],
#'max_depth' : [10,20,30],
#'max_features' : [5,8,10,12],
 #    }


from sklearn.model_selection import RandomizedSearchCV



In [26]:
#random_cv=RandomizedSearchCV(estimator= rfr, param_distributions= ss, 
 #                           cv=5, n_iter=50, 
  #                          scoring= 'neg_mean_squared_error',
   #                         n_jobs=4, verbose =5, 
    #                        return_train_score = True,
     #                         random_state= 0)

In [27]:
#random_cv.fit(X_train, y_train)

In [28]:
#random_cv.best_estimator_

In [29]:
#import xgboost 
#xgbr=xgboost.XGBRegressor()
#xgbr.fit(X_train, y_train)
#pred_xgbr=xgbr.predict(X_train)
#mse=mean_squared_error(y_train, pred_xgbr)
#mse
#rmse=np.sqrt(mse)
#rmse

In [30]:
rfr=RandomForestRegressor(random_state=0,  n_estimators= 500, max_depth= 30, max_features=5, warm_start= False)
rfr.fit(X_train, y_train)
pred_rfr=rfr.predict(X_train)
mse=mean_squared_error(y_train, pred_rfr, squared=True)
mse
rmse=np.sqrt(mse)
rmse


130.29157380417146

# Thanks for Reading:)